In [135]:
import pandas as pd
import re
import difflib

In [136]:
# Movie Info
df_movielen_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/movies.csv")
df_movielen_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Kết quả Schema Matching: (Những trường bị đánh * trong Source Schema là bị matching sai)

- Bảng Movie

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| movieId*      | runtime               |
| title         | movie_title           |
| genres        | genres                |

- Bảng Rating: như nhận xét, kết quả matching sai hoàn toàn

Nhận xét và hướng xử lý của các trường:

- Bảng Movie:
    +  Ở đây ta thấy title trong dữ liệu có cấu trúc là tên phim + (năm ra mắt phim) => tách trường title thành 2 trường title và year
    + Cấu trúc giá trị của genres theo như mô tả trong data warehouse là <genres 1>, <genres 2>, ... , trong khi đó ở trong MovieLen, cấu trúc của genres lại là <genres 1>|<genres 2>|... , đồng thời các thể loại trong MovieLen có thể khác so với thể loại trong Data Warehouse => cần phải biến đổi lại
    + Các thông tin về movie_info, production_company, directors, actors, runtime, content_rating không có sẵn trong nguồn MovieLen

- Bảng Rating:
    + Về các thông tin của avg_rating, rating_count, ta sẽ cập nhật từ dữ liệu rating của MovieLen
    + Thông tin về link phim không có sẵn trong nguồn MovieLen


# Xử lý trường title

In [137]:
# Split year
import re
df_movielen_movie["year"]=[re.findall(r'(\(\d{4}\))', i) for i in df_movielen_movie["title"]]

In [138]:
count=0
print("Những trường hợp ngoại lệ:")
for idx, i in enumerate(df_movielen_movie["year"]):
    if len(i)==1:
        count+=1
    if len(i)>1:
        print("Title: "+df_movielen_movie.iloc[idx]["title"])
print("Số bản ghi có chứa năm ra mắt: "+str(count))

Những trường hợp ngoại lệ:
Title: The Devotion of Suspect X (2017) (2017)
Title: Don Quixote (1973) (1973)
Số bản ghi có chứa năm ra mắt: 62011


Ta thấy những trường hợp ngoại lệ vẫn có thể suy ra được năm phim ra mắt

In [139]:
def year_preprocess(year_list):
    if len(year_list)>=1:
        # Loại bỏ dấu ngoặc đơn
        return int(year_list[0][1:-1])
    else:
        return None
df_movielen_movie["year"]=[year_preprocess(i) for i in df_movielen_movie["year"]]
# Loại bỏ năm trong title
df_movielen_movie["title"]=[re.sub(r'(\(\d{4}\))', '', i) for i in df_movielen_movie["title"]]

In [140]:
df_movielen_movie.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


# Xử lý trường genres

In [141]:
# Chuyển string thành list
df_movielen_movie["genres"]=[i.split("|") for i in df_movielen_movie["genres"]]
for i in df_movielen_movie["genres"]:
    if len(i)==0:
        count+=1
print("Số bản ghi mà trường genres bị rỗng: "+str(count))

Số bản ghi mà trường genres bị rỗng: 62011


In [142]:
# Lấy danh sách các thể loại có trong MovieLen
movielen_genres_list = []
for i in df_movielen_movie["genres"]:
    movielen_genres_list+=i
movielen_genres_list = list(set(movielen_genres_list))
movielen_genres_list

['Mystery',
 'Children',
 'Western',
 'Horror',
 'Drama',
 'Action',
 'Documentary',
 'War',
 'Romance',
 'Crime',
 'Musical',
 '(no genres listed)',
 'Film-Noir',
 'Comedy',
 'Animation',
 'Thriller',
 'IMAX',
 'Sci-Fi',
 'Fantasy',
 'Adventure']

Ta nhận thấy trong thể loại của MovieLen có giá trị '(no genres listed)', ta tiến hành kiểm tra giá trị đó xem nó xuất hiện trong các bản ghi như thế nào

In [143]:
count=0
print("Các trường hợp ngoại lệ")
for idx, i in enumerate(df_movielen_movie["genres"]):
    if '(no genres listed)' in i:
        count+=1
        if len(i)>1:
            print(i)
print("Số bản ghi có giá trị (no genres listed) là: "+str(count))

Các trường hợp ngoại lệ
Số bản ghi có giá trị (no genres listed) là: 5062


Có thể thấy rằng những bộ phim bị gắn thể loại (no genres listed) là chưa rõ về thể loại, và không có trường hợp ngoại lệ ((no genres listed) đi cùng với 1 hay nhiều thể loại khác) => ta có thể thay giá trị này thành null

In [144]:
# Loại bỏ giá trị (no genres listed)
remove_idx = movielen_genres_list.index("(no genres listed)")
del movielen_genres_list[remove_idx]

In [145]:
warehouse_genres_list = list(pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv")["genres"])
warehouse_genres_list

['Kids & Family',
 'Horror',
 'Anime & Manga',
 'Gay & Lesbian',
 'Drama',
 'Special Interest',
 'Comedy',
 'Documentary',
 'Romance',
 'Cult Movies',
 'Musical & Performing Arts',
 'Faith & Spirituality',
 'Sports & Fitness',
 'Art House & International',
 'Mystery & Suspense',
 'Western',
 'Classics',
 'Science Fiction & Fantasy',
 'Animation',
 'Action & Adventure',
 'Television']

Từ 2 danh sách thể loại, ta có thể mapping như dưới:
|Data warehouse Genres|MovieLen Genres |
| :- | :- |
|‘Classic’||
|‘Documentary’|‘Documentary’|
|‘Western’|‘Western’|
|‘Horror’|‘Horror’|
|‘Science Fiction & Fantasy’ *|‘Sci-Fi’, ‘Fantasy’|
|‘Gay & Lesbian’||
|‘Drama’|‘Drama’|
|‘Comedy’|‘Comedy’|
|‘Cult Movies’||
|‘Romance’|‘Romance’|
|‘Television’||
|‘Sport & Fitness’||
|‘Art House & International’||
|‘Special Interest’||
|‘Animation’|‘Animation’|
|‘Musical & Performing Arts’ *|‘Musical’|
|‘Faith & Spirituality’||
|‘Mystery & Suspense’ *|‘Mystery’|
|‘Anime & Manga’||
|‘Action & Adventure’ *|‘Action’, ‘Adventure’|
|‘Kids & Family’ *|‘Children’|

Những giá trị được đánh * là những giá trị cần phải chuyển đổi

Những giá trị như War, Crime, Thrill, Film-Noir, IMAX ta sẽ thêm vào Data Warehouse

In [146]:
replace_genres_dict = {'Sci-Fi': 'Science Fiction & Fantasy', 'Fantasy': 'Science Fiction & Fantasy',
                       'Musical': 'Musical & Performing Arts', 'Mystery': 'Mystery & Suspense', 
                       'Action': 'Action & Adventure', 'Adventure': 'Action & Adventure', 'Children': 'Kids & Family', }
def preprocess_genres(list_genres):
    if '(no genres listed)' in list_genres:
        return None
    else:
        new_list_genres = ""
        for idx, i in enumerate(list_genres):
            if i in replace_genres_dict:
                new_list_genres+=replace_genres_dict[i]
            else:
                new_list_genres+=i
            if idx != len(list_genres)-1:
                new_list_genres+=", "
        return new_list_genres
df_movielen_movie["genres"] = [preprocess_genres(i) for i in df_movielen_movie["genres"]]

In [147]:
# Thêm các giá trị mới vào trong genres warehouse
warehouse_genres_list+=["War", "Crime", "Thrill", "Film-Noir", "IMAX"]
warehouse_genres_list = list(set(warehouse_genres_list))
pd.DataFrame(warehouse_genres_list, columns=["genres"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv",
    index=False 
)

# Tạo thêm trường avg_rating và rating_count

Ta tổng hợp lại rating bằng cách tính avgrating và ratecount trong bảng df_movielen_rating 

In [148]:
df_movielen_rating = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/ratings.csv")
df_movielen_statistic = pd.DataFrame()
df_movielen_statistic["movieId"] = df_movielen_rating.groupby("movieId")["rating"].mean().keys()
df_movielen_statistic["avgrating"] = df_movielen_rating.groupby("movieId")["rating"].mean().values
df_movielen_statistic["ratecount"] = df_movielen_rating.groupby("movieId")["rating"].count().values
df_movielen_statistic.head()

,movieId,avgrating,ratecount
0,1,3.893708,57309
1,2,3.251527,24228
2,3,3.142028,11804
3,4,2.853547,2523
4,5,3.058434,11714


In [149]:
df_movielen_final = pd.concat([df_movielen_movie, df_movielen_statistic], axis=1)
df_movielen_final

,movieId,title,genres,year,movieId,avgrating,ratecount
0,1,Toy Story,"Action & Adventure, Animation, Kids & Family, ...",1995.0,1.0,3.893708,57309.0
1,2,Jumanji,"Action & Adventure, Kids & Family, Science Fic...",1995.0,2.0,3.251527,24228.0
2,3,Grumpier Old Men,"Comedy, Romance",1995.0,3.0,3.142028,11804.0
3,4,Waiting to Exhale,"Comedy, Drama, Romance",1995.0,4.0,2.853547,2523.0
4,5,Father of the Bride Part II,Comedy,1995.0,5.0,3.058434,11714.0
...,...,...,...,...,...,...,...
62418,209157,We,Drama,2018.0,NaN,NaN,NaN
62419,209159,Window of the Soul,Documentary,2001.0,NaN,NaN,NaN
62420,209163,Bad Poems,"Comedy, Drama",2018.0,NaN,NaN,NaN
62421,209169,A Girl Thing,None,2001.0,NaN,NaN,NaN


Ta thấy có 2 cột movieId đang bị lặp, ta sẽ chỉ giữ lại 1 cột là cột không chứa giá trị null nào

In [150]:
cols=pd.Series(df_movielen_final.columns)
for dup in df_movielen_final.columns[df_movielen_final.columns.duplicated(keep=False)]: 
    cols[df_movielen_final.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(df_movielen_final.columns.get_loc(dup).sum())]
                                    )
df_movielen_final.columns=cols
df_movielen_final

,movieId,title,genres,year,movieId.1,avgrating,ratecount
0,1,Toy Story,"Action & Adventure, Animation, Kids & Family, ...",1995.0,1.0,3.893708,57309.0
1,2,Jumanji,"Action & Adventure, Kids & Family, Science Fic...",1995.0,2.0,3.251527,24228.0
2,3,Grumpier Old Men,"Comedy, Romance",1995.0,3.0,3.142028,11804.0
3,4,Waiting to Exhale,"Comedy, Drama, Romance",1995.0,4.0,2.853547,2523.0
4,5,Father of the Bride Part II,Comedy,1995.0,5.0,3.058434,11714.0
...,...,...,...,...,...,...,...
62418,209157,We,Drama,2018.0,NaN,NaN,NaN
62419,209159,Window of the Soul,Documentary,2001.0,NaN,NaN,NaN
62420,209163,Bad Poems,"Comedy, Drama",2018.0,NaN,NaN,NaN
62421,209169,A Girl Thing,None,2001.0,NaN,NaN,NaN


In [151]:
df_movielen_final.drop(columns=['movieId.1'], inplace=True)

In [152]:
df_movielen_final.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/movies_preprocess.csv", index=False)